# 1. Instalación de librerias en python

In [0]:
!pip install pandas
!pip install pandas_datareader
!pip install datetime
!pip install matplotlib

# 2. Habilitando las librerias

In [0]:
import pandas as pd
import pandas_datareader.data as web   # Importacion de las libreias
from pandas_datareader import data
import datetime
import matplotlib.pyplot as plt
import numpy as np

# 3. Lectura de los datos

In [0]:
# Fecha de información
start = datetime.datetime(2017,1,1)
end = datetime.date.today()
#end = datetime.datetime(2016,12,31)

microsoft = web.DataReader("MSFT", "yahoo", start, end)
google = web.DataReader("GOOG", "yahoo", start, end)
gsp = web.DataReader("^GSPC", "yahoo", start, end)

# Lectura del precio de las acciones de Apple desde Yahoo Finance!
stocks = pd.DataFrame({"GOOG": google["Adj Close"],
                      "MSFT": microsoft["Adj Close"],
                      "^GSPC": gsp["Adj Close"]})



In [0]:
type(stocks)

In [0]:
stocks.head(9)

In [0]:
stocks.shape

In [0]:
stocks.describe()

In [0]:
# Graficando el valor de las acciones
#-----------------------------------------------
stocks.plot(grid = True)

In [0]:
# Configuracion de revision mensual #3M y #D
all_weekdays = pd.date_range(start=start, end=end, freq='M')

close = stocks.reindex(all_weekdays)
close = close.fillna(method='ffill')

In [0]:
close.shape

In [0]:
close.describe()

In [0]:
# Graficando el valor de las acciones
#-----------------------------------------------
close.plot(grid = True)

# 4. Analisis de Tendencia: 
# Graficas de medias moviles Rolling

In [0]:
# Selecionamos una columna de precios.
msft = stocks.loc[:, 'MSFT']

# Medias Moviles de corto plazo (20) y largo plazo (100)
short_rolling_msft = msft.rolling(window=20).mean()
long_rolling_msft = msft.rolling(window=100).mean()
#mid_rolling_msft = msft.rolling(window=50).mean()

# Agregando el grafico con las medis moviles
fig, ax = plt.subplots(figsize=(16,9))

ax.plot(msft.index, msft, label='MSFT')
ax.plot(short_rolling_msft.index, short_rolling_msft, label='20 days rolling')
ax.plot(long_rolling_msft.index, long_rolling_msft, label='100 days rolling')
#ax.plot(mid_rolling_msft.index, mid_rolling_msft, label='50 days rolling')

ax.set_xlabel('Date')
ax.set_ylabel('Adjusted closing price ($)')
ax.legend()

# 5. Analisis de Volatilidad:
# Bandas de Bollinger

In [0]:
# Indicadores de volatilidad

msft_mean = msft.rolling(window=20).mean()
msft_sd = msft.rolling(window=20).std()
msft_up = msft_mean + (msft_sd * 2)
msft_low = msft_mean - (msft_sd * 2)
    
# Graficando las bandas de volatilidad
fig, ax = plt.subplots(figsize=(16,9))

ax.plot(msft.index, msft, label='MSFT',color='blue', lw=1)
ax.plot(msft_up.index, msft_up, label='Lim Sup',color='green', lw=1)
ax.plot(msft_low.index, msft_low, label='Lim Inf',color='green', lw=1)

#Etiquetas y leyendas

ax.set_xlabel('Date')
ax.set_ylabel('Adjusted closing price ($)')
ax.legend()


#https://medium.com/python-data/setting-up-a-bollinger-band-with-python-28941e2fa300


# 6. Analisis de Oscilacion o Fuerza del precio
# Indicador RSI

In [0]:
# Si el RSI alcanza el nivel de 70 hay que interpretar que la accion está sobrecomprado (cuyo significado es que el especulador debería plantearse vender).
# Y análogamente al contrario, sobrevendido en el nivel de 30.
# La hipótesis sería que cuando hay una alta proporción de 
# movimientos diarios en una dirección esto sugiere que se alcanzará el extremo, y los precios tenderán a darse la vuelta.

#Teoria de RSI
# https://www.21tradingcoach.com/es/formaci%C3%B3n-gratuita/an%C3%A1lisis-t%C3%A9cnico/165-indicador-rsi-relative-strength-index

#Libreri Tab Lib
# https://towardsdatascience.com/trading-strategy-technical-analysis-with-python-ta-lib-3ce9d6ce5614


In [0]:

def rsiFunc(prices, n=14):
    deltas = np.diff(prices)
    seed = deltas[:n+1]
    up = seed[seed>=0].sum()/n
    down = -seed[seed<0].sum()/n
    rs = up/down
    rsi = np.zeros_like(prices)
    rsi[:n] = 100. - 100./(1.+rs)

    for i in range(n, len(prices)):
        delta = deltas[i-1] # cause the diff is 1 shorter

        if delta>0:
            upval = delta
            downval = 0.
        else:
            upval = 0.
            downval = -delta

        up = (up*(n-1) + upval)/n
        down = (down*(n-1) + downval)/n

        rs = up/down
        rsi[i] = 100. - 100./(1.+rs)

    return rsi



In [0]:
df=msft
msft_rsi=rsiFunc(df)

In [0]:
# Graficando el valor de la accion
fig, ax = plt.subplots(figsize=(16,9))
ax.plot(msft.index, msft, label='MSFT',color='blue', lw=1)


# Graficando el  RSI o indicador del volumne de negociacion
fig, ax2 = plt.subplots(figsize=(16,9))
ax2.plot(msft.index, msft_rsi, label='MSFT',color='blue', lw=1)




In [0]:
# Librerias disponibles para Velas Japonesas
import plotly.plotly as py
import plotly.graph_objs as go

In [0]:
# Extrayendo los 4 datos del precio de laccion de Microsoft

start = datetime.datetime(2017,1,1)
end = datetime.date.today()
#end = datetime.datetime(2016,12,31)

df = web.DataReader("MSFT", "yahoo", start, end)


In [0]:
df.head(5)

In [0]:
# Grafica de Vela Japonesa

trace = go.Candlestick(x=df.index,
                open=df.Open,
                high=df.High,
                low=df.Low,
                close=df.Close)
data = [trace]
py.iplot(data, filename='simple_candlestick')

